# Inicialization


In [68]:
# Bibliotecas

from google.colab import drive
import os
import cv2
import random
import numpy as np
import hashlib
from collections import defaultdict
import pathlib
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [49]:
# Montar o Google Drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [50]:
# Paths

malignant_directory = '/content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Database/Malignant/descomp'
benign_directory = '/content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Database/Benign/descomp'
mix_directory = '/content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Preprocessing/Identification/Mix'
resize_directory = '/content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Preprocessing/Resizing'
mask_directory = '/content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Preprocessing/Segmentation/Mask'
segmented_directory = '/content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Preprocessing/Segmentation/Segmented'
reduced_directory = '/content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Extraction/Reduction'
hog_directory = '/content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Extraction/HoG'
labels_directory = '/content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Training/Labels'

In [74]:
# Variáveis Globais

total_images = len([file for file in os.listdir(mix_directory) if file.lower().endswith('.jpg')])
total_images_ma = len([file for file in os.listdir(malignant_directory) if file.lower().endswith('.jpg')])
total_images_be = len([file for file in os.listdir(benign_directory) if file.lower().endswith('.jpg')])
total_images_mix = len([file for file in os.listdir(mix_directory) if file.lower().endswith('.jpg')])

train_size = 0.3
reduction_factor = 0.8
image_size = (608, 424)
batch_sizes_to_evaluate = [10, 20, 30, 40, 50]

In [52]:
# Número total de imagens

print(total_images_ma)
print(total_images_be)
print(total_images_mix)

171
729
900


# Preprocessing

In [23]:
# Remover Duplicatas

def get_file_hash(file_path):
    hasher = hashlib.sha1()
    with open(file_path, 'rb') as f:
        while chunk := f.read(8192):
            hasher.update(chunk)
    return hasher.hexdigest()

def find_duplicates(directory):
    file_hash_dict = defaultdict(list)
    for file_name in os.listdir(directory):
        if file_name.lower().endswith('.jpg'):
            file_path = os.path.join(directory, file_name)
            file_hash = get_file_hash(file_path)
            file_hash_dict[file_hash].append(file_path)

    return {hash_value: file_list for hash_value, file_list in file_hash_dict.items() if len(file_list) > 1}

malignant_duplicates = find_duplicates(malignant_directory)
benign_duplicates = find_duplicates(benign_directory)
input_duplicates = find_duplicates(mix_directory)

print("Duplicated files in Malignant directory:")
for hash_value, file_list in malignant_duplicates.items():
    print(f"Hash: {hash_value}, Files: {file_list}")

print("\nDuplicated files in Benign directory:")
for hash_value, file_list in benign_duplicates.items():
    print(f"Hash: {hash_value}, Files: {file_list}")

print("\nDuplicated files in Input directory:")
for hash_value, file_list in input_duplicates.items():
    print(f"Hash: {hash_value}, Files: {file_list}")



Duplicated files in Malignant directory:
Hash: 66fb4f55f62841bd2ba322f3e985b764f83e6444, Files: ['/content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Database/Malignant/descomp/150.JPG', '/content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Database/Malignant/descomp/72.JPG']

Duplicated files in Benign directory:
Hash: 311e59f77b9bd66ab66c42e1880859567ac55af0, Files: ['/content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Database/Benign/descomp/574.JPG', '/content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Database/Benign/descomp/537.JPG']

Duplicated files in Input directory:
Hash: 66fb4f55f62841bd2ba322f3e985b764f83e6444, Files: ['/content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Preprocessing/Identification/Mix/150.JPG', '/content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/

In [24]:
# Redimensionamento das imagens

def resize_images(input_directory, output_directory, size):
    # Obter a lista de arquivos no diretório de entrada
    image_files = [file for file in os.listdir(input_directory) if file.lower().endswith(('.jpg'))]

    # Contar o número total de imagens
    total_images = len(image_files)

    # Redimensionamento das imagens
    for i in range(1, total_images + 1):
        # Nome do arquivo de entrada
        input_filename = f'{i}.jpg'  # ou a extensão correspondente dos seus arquivos
        input_path = os.path.join(input_directory, input_filename)

        # Se o arquivo não existir com a extensão .jpg, tentar com .JPG (necessário pois )
        if not os.path.isfile(input_path):
            input_filename = f'{i}.JPG'
            input_path = os.path.join(input_directory, input_filename)

        # Verificar se o arquivo existe
        if os.path.isfile(input_path):
            # Carregar a imagem
            img = cv2.imread(input_path)

            # Verificar se a imagem foi carregada corretamente
            if img is not None:
                # Redimensionar a imagem
                img_resized = cv2.resize(img, size)

                # Nome do arquivo de saída
                output_filename = f'{i}_resized.jpg'
                output_path = os.path.join(output_directory, output_filename)

                # Salvar a imagem redimensionada
                cv2.imwrite(output_path, img_resized)

                print(f'Imagem {i} redimensionada e salva em {output_path}')
            else:
                print(f'Erro ao carregar a imagem {i} em {input_path}. Verifique se o arquivo está no formato correto.')
        else:
            print(f'Arquivo {input_path} não encontrado. Verifique se os arquivos estão na pasta correta.')

    print('Redimensionamento concluído.')


In [25]:
resize_images(mix_directory, resize_directory, image_size)

Imagem 1 redimensionada e salva em /content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Preprocessing/Resizing/1_resized.jpg
Imagem 2 redimensionada e salva em /content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Preprocessing/Resizing/2_resized.jpg
Imagem 3 redimensionada e salva em /content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Preprocessing/Resizing/3_resized.jpg
Imagem 4 redimensionada e salva em /content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Preprocessing/Resizing/4_resized.jpg
Imagem 5 redimensionada e salva em /content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Preprocessing/Resizing/5_resized.jpg
Imagem 6 redimensionada e salva em /content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Preprocessing/Resizing/6_resized.jpg
Imagem 7 redimensionada e salva em

In [26]:
# Criar Máscaras binarizadas

def create_binary_masks(input_directory, output_directory, total_images):
    os.makedirs(output_directory, exist_ok=True)

    for i in range(1, total_images + 1):
        input_filename = f'{i}_resized.jpg'
        input_path = os.path.join(input_directory, input_filename)

        if os.path.isfile(input_path):
            img_resized = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)
            _, mask_adaptive = cv2.threshold(img_resized, 128, 255, cv2.THRESH_BINARY_INV)
            kernel = np.ones((2, 2), np.uint8)
            cleaned_mask = cv2.morphologyEx(mask_adaptive, cv2.MORPH_OPEN, kernel)
            output_mask_filename = f'{i}_mask.jpg'
            output_mask_path = os.path.join(output_directory, output_mask_filename)
            cv2.imwrite(output_mask_path, cleaned_mask)
            print(f'Máscara {i} (Limiar Adaptativo) salva em {output_mask_path}')
        else:
            print(f'Arquivo {input_path} não encontrado. Verifique se os arquivos estão na pasta correta.')

In [28]:
create_binary_masks(resize_directory, mask_directory, total_images)

Máscara 1 (Limiar Adaptativo) salva em /content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Preprocessing/Segmentation/Mask/1_mask.jpg
Máscara 2 (Limiar Adaptativo) salva em /content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Preprocessing/Segmentation/Mask/2_mask.jpg
Máscara 3 (Limiar Adaptativo) salva em /content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Preprocessing/Segmentation/Mask/3_mask.jpg
Máscara 4 (Limiar Adaptativo) salva em /content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Preprocessing/Segmentation/Mask/4_mask.jpg
Máscara 5 (Limiar Adaptativo) salva em /content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Preprocessing/Segmentation/Mask/5_mask.jpg
Máscara 6 (Limiar Adaptativo) salva em /content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Preprocessing/Seg

# Segmentation

In [29]:
# Aplicação de máscaras

def apply_masks(input_directory, mask_directory, output_directory, total_images):
    os.makedirs(output_directory, exist_ok=True)

    for i in range(1, total_images + 1):
        input_filename = f'{i}_resized.jpg'
        input_path = os.path.join(input_directory, input_filename)
        mask_filename = f'{i}_mask.jpg'
        mask_path = os.path.join(mask_directory, mask_filename)

        if os.path.isfile(input_path) and os.path.isfile(mask_path):
            img_resized = cv2.imread(input_path)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            segmented_image = cv2.bitwise_and(img_resized, img_resized, mask=mask)
            output_segmented_filename = f'{i}_segmented.jpg'
            output_segmented_path = os.path.join(output_directory, output_segmented_filename)
            cv2.imwrite(output_segmented_path, segmented_image)
            print(f'Imagem {i} segmentada e salva em {output_segmented_path}')
        else:
            print(f'Arquivos para a imagem {i} não encontrados. Verifique se os arquivos estão na pasta correta.')

In [30]:
apply_masks(resize_directory, mask_directory, segmented_directory, total_images)

Imagem 1 segmentada e salva em /content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Preprocessing/Segmentation/Segmented/1_segmented.jpg
Imagem 2 segmentada e salva em /content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Preprocessing/Segmentation/Segmented/2_segmented.jpg
Imagem 3 segmentada e salva em /content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Preprocessing/Segmentation/Segmented/3_segmented.jpg
Imagem 4 segmentada e salva em /content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Preprocessing/Segmentation/Segmented/4_segmented.jpg
Imagem 5 segmentada e salva em /content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Preprocessing/Segmentation/Segmented/5_segmented.jpg
Imagem 6 segmentada e salva em /content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Preprocessing/S

In [37]:
# Redução de tamanho para 20% das imagens redimensionadas

def reduce_images(input_directory, output_directory, reduction_factor):
    os.makedirs(output_directory, exist_ok=True)

    for i in range(1, total_images + 1):
        input_filename = f'{i}_resized.jpg'
        input_path = os.path.join(input_directory, input_filename)

        if os.path.isfile(input_path):
            img_resized = cv2.imread(input_path)
            height, width = img_resized.shape[:2]
            img_reduced = cv2.resize(img_resized, (int(width * (1 - reduction_factor)), int(height * (1 - reduction_factor))))
            output_reduced_filename = f'{i}_reduced.jpg'
            output_reduced_path = os.path.join(output_directory, output_reduced_filename)
            cv2.imwrite(output_reduced_path, img_reduced)
            print(f'Imagem {i} reduzida e salva em {output_reduced_path}')
        else:
            print(f'Arquivo {input_path} não encontrado. Verifique se os arquivos estão na pasta correta.')

In [38]:
reduce_images(resize_directory, reduced_directory, reduction_factor)

Imagem 1 reduzida e salva em /content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Extraction/Reduction/1_reduced.jpg
Imagem 2 reduzida e salva em /content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Extraction/Reduction/2_reduced.jpg
Imagem 3 reduzida e salva em /content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Extraction/Reduction/3_reduced.jpg
Imagem 4 reduzida e salva em /content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Extraction/Reduction/4_reduced.jpg
Imagem 5 reduzida e salva em /content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Extraction/Reduction/5_reduced.jpg
Imagem 6 reduzida e salva em /content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Extraction/Reduction/6_reduced.jpg
Imagem 7 reduzida e salva em /content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/

# Extraction

In [39]:
# Extração de descritores HoG

def extract_hog_features(input_directory, output_directory):
    os.makedirs(output_directory, exist_ok=True)

    win_size = (16, 16)
    block_size = (8, 8)
    block_stride = (4, 4)
    cell_size = (4, 4)
    nbins = 8

    for i in range(1, total_images + 1):
        input_filename = f'{i}_reduced.jpg'
        input_path = os.path.join(input_directory, input_filename)

        if os.path.isfile(input_path):
            img_reduced = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)
            hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, nbins)
            hog_features = hog.compute(img_reduced)
            output_hog_filename = f'{i}_hog.npy'
            output_hog_path = os.path.join(output_directory, output_hog_filename)

            with open(output_hog_path, 'wb') as f:
                np.save(f, hog_features)

            print(f'Descritores HoG da imagem {i} salvos em {output_hog_path}')
        else:
            print(f'Arquivo {input_path} não encontrado. Verifique se os arquivos estão na pasta correta.')

In [40]:
extract_hog_features(reduced_directory, hog_directory)

Descritores HoG da imagem 1 salvos em /content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Extraction/HoG/1_hog.npy
Descritores HoG da imagem 2 salvos em /content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Extraction/HoG/2_hog.npy
Descritores HoG da imagem 3 salvos em /content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Extraction/HoG/3_hog.npy
Descritores HoG da imagem 4 salvos em /content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Extraction/HoG/4_hog.npy
Descritores HoG da imagem 5 salvos em /content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Extraction/HoG/5_hog.npy
Descritores HoG da imagem 6 salvos em /content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Extraction/HoG/6_hog.npy
Descritores HoG da imagem 7 salvos em /content/drive/Othercomputers/PC_Mirror/Mirror/UTF

# Training

As Labels são geradas baseado na numeração e pasta do imagem, porém o tamanho das pastas e numeração é calculado anteriormente na etapa de inicialização, onde as imagens em "Mix" já foram ordenadas através de um script em bash. Assim esta é uma função generalista, pois caso fossem adicionados mais arquivos ao banco, ocorreria uma reidentificação das imagens.
Ex:

Banco atual:
total_images = 900|total_images_ma = 171|total_images_be = 729
Labels: 1 a 171 = 1| 172 a 900 = 0

Banco novo (+1 imagem maligna): total_images = 901|total_images_ma = 172|total_images_be = 729
Labels: 1 a 172 = 1| 173 a 901 = 0

In [41]:
# Gerar Label

def generate_labels(benign_directory, malignant_directory, labels_directory):

    # Criar o diretório de labels se não existir
    os.makedirs(labels_directory, exist_ok=True)

    # Criar e salvar os labels
    for i in range(1, total_images + 1):
        if i <= total_images_ma:
            # Imagem maligna (melanoma)
            label = '1'
        else:
            # Imagem benigna
            label = '0'

        # Salvar o label em um arquivo de texto
        label_filename = f'{i}_label.txt'
        label_path = os.path.join(labels_directory, label_filename)

        with open(label_path, 'w') as file:
            file.write(label)

    print('Labels criados e salvos em', labels_directory)

In [42]:
generate_labels(benign_directory, malignant_directory, labels_directory)

Labels criados e salvos em /content/drive/Othercomputers/PC_Mirror/Mirror/UTFPR/Períodos/4 2023_02/PI/Projeto Final/Training/Labels


In [45]:
# Executar etapa de treinamento e avaliação do SVM

def extract_and_train_svm(hog_directory, labels_directory, test_size, random_state):
    hog_data = []
    labels = []

    for i in range(1, total_images + 1):
        hog_filename = f'{i}_hog.npy'
        hog_path = os.path.join(hog_directory, hog_filename)
        hog_descriptor = np.load(hog_path)
        hog_data.append(hog_descriptor)

        label_filename = f'{i}_label.txt'
        label_path = os.path.join(labels_directory, label_filename)

        with open(label_path, 'r') as file:
            label = int(file.read())
            labels.append(label)

    hog_data = np.array(hog_data)
    labels = np.array(labels)

    X_train, X_test, y_train, y_test = train_test_split(hog_data, labels, test_size= 0.3, random_state= 42, stratify=labels)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    svm_model = SVC(kernel='linear', C=1)
    svm_model.fit(X_train, y_train)

    y_pred = svm_model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    print(f'Acurácia: {accuracy * 100:.2f}%')

In [46]:
extract_and_train_svm(hog_directory, labels_directory, train_size, 42)

Acurácia: 77.78%


# Evaluation

In [70]:
def evaluate_svm(hog_directory, labels_directory, batch_sizes, test_size):
    random_state = int(0)
    results = []

    for batch_size in batch_sizes:
        print(f'Treinando e avaliando SVM para batch_size = {batch_size}...')

        hog_data = []
        labels = []

        for i in range(1, total_images + 1):
            hog_filename = f'{i}_hog.npy'
            hog_path = os.path.join(hog_directory, hog_filename)
            hog_descriptor = np.load(hog_path)
            hog_data.append(hog_descriptor)

            label_filename = f'{i}_label.txt'
            label_path = os.path.join(labels_directory, label_filename)

            with open(label_path, 'r') as file:
                label = int(file.read())
                labels.append(label)

        hog_data = np.array(hog_data)
        labels = np.array(labels)

        X_train, X_test, y_train, y_test = train_test_split(hog_data, labels, test_size=test_size, random_state=random.randint(1,50), stratify=labels)

        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        svm_model = SVC(kernel='linear', C=1)
        svm_model.fit(X_train, y_train)

        y_pred = svm_model.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        results.append((batch_size, accuracy * 100))

    # Criar DataFrame com os resultados
    columns = ['Batch Size', 'Accuracy']
    df_results = pd.DataFrame(results, columns=columns)

    # Encontrar o melhor resultado
    best_result = df_results[df_results['Accuracy'] == df_results['Accuracy'].max()]

    print('\nResultados:')
    print(df_results.to_string(index=False))
    print(f'\nMelhor resultado:\n{best_result.to_string(index=False)}')

In [71]:
evaluate_svm(hog_directory, labels_directory, batch_sizes_to_evaluate, train_size)

Treinando e avaliando SVM para batch_size = 10...
Treinando e avaliando SVM para batch_size = 20...
Treinando e avaliando SVM para batch_size = 30...
Treinando e avaliando SVM para batch_size = 40...
Treinando e avaliando SVM para batch_size = 50...

Resultados:
 Batch Size  Accuracy
         10 81.481481
         20 80.000000
         30 79.629630
         40 80.370370
         50 79.629630

Melhor resultado:
 Batch Size  Accuracy
         10 81.481481
